## 0 Intro

本 note 关注于实现神经网络中常用的层，设计原则如下：

1. 每一层均使用一个类来实现
2. 每一个类维护前向计算时所需要的**参数**、后向计算时所需要的**中间结果**、更新参数时所需要的**梯度**
3. 每个类的**参数**和**梯度**用列表 `params` 和 `grads` 打包，便于整个神经网络的训练
4. 每一个类至少包含三个方法，即 `__init__()`、`forward()`、`backward()`
5. 复杂类可以基于简单类来实现

**备注**
> 梯度的形状和对应参数的形状一致

## 1 MatMul Layer

本层执行矩阵乘法运算，即 $\mathbf{y} = \mathbf{x}\mathbf{W}$，这里不考虑偏置值

实现如下：

**成员变量**
- 参数： `W` - 参数矩阵
- 中间结果 `x` - 输入数据
- 梯度: `dW`

**成员函数**

- `__init__(self, W)`
- `forward(self, x)`
- `backward(self, dout)`

代码如下：

In [ ]:
class MatMul:
    def __init__(self, W):
        '''初始化所需要维护的成员变量，其中参数 W 由外界传入
        '''
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        '''前向计算
        
        :param x: 是输入数据
        
        :return: 输出矩阵乘法的结果
        '''
        W, = self.params
        self.x = x
        out = np.dot(x, W)
        
        return out
    
    def backward(self, dout):
        '''后向计算，计算更新本层参数的梯度，以及传播到后一层的导数
        
        :param dout: 上游来的导数信号
        
        :return dx: 传给下游的导数信号
        '''
        ## 这里考虑了 x 是批数据的情况
        W, = self.params
        dW = np.dot(self.x.T, dout)
        dx = np.dot(dout, W.T)
        
        self.grads[0][...] = dW  # 注意这里的深拷贝
        
        return dx

## 2 Affine Layer

Affine 层实现的仿射变换，即 $\mathbf{y}=\mathbf{x}\mathbf{W} + \mathbf{b}$，相比 MatMul，Affine 多了一个偏置值的计算

实现如下：

**成员变量**

- 参数：`W` - 矩阵； `b` - 偏置值;
- 中间结果: `x` - 输入数据
- 梯度: `dW`；`db`

**成员函数**

- `__init__(self, W, b)`
- `forward(self, x)`
- `backward(self, dout)`

代码如下：

In [ ]:
class Affine:
    def __init__(self, W, b):
        '''初始化 Affine 所维护的参数，参数 W 和 b 由外界传入
        '''
        self.params = [W, b]  # 参数列表
        self.grads = [np.zeros_like(W), np.zeros_like(b)]  # 梯度列表
        self.x = None
        
    def forward(self, x):
        '''前向计算
        '''
        W, b = self.params
        self.x = x
        out = np.dot(x, W) + b
        
        return out
    
    def backward(self, dout):
        '''后向计算，计算更新本层参数所需的梯度，以及下游所需的导数信号
        '''
        W, b = self.params
        
        dW
        db
        dx = s